# Multi frequency deconvolution with Rascil and Radler

This script runs a multifrequency deconvolution in Rascil and Radler using a pre-existing dataset

In [ ]:
%matplotlib inline

import os
import sys

sys.path.append(os.path.join('..', '..'))

from matplotlib import pylab

pylab.rcParams['image.cmap'] = 'rainbow'

import numpy

from astropy.io import fits

from matplotlib import pyplot as plt

from rascil.processing_components import import_image_from_fits, \
    deconvolve_cube, restore_cube

import radler as rd


The following block reads multi-frequency dataset.
The dataset is generated with the following wsclean command: 

"wsclean -size 1024 1024 -scale 1amin -channels-out 8 -make-psf MWA-single-timeslot.ms"

In [ ]:
fits_path =  "/var/scratch/csalvoni/data/mwa/wsclean-000"
dirty_np = fits.open(fits_path + "0-dirty.fits")[0].data
psf_np = fits.open(fits_path + "0-psf.fits")[0].data

for i in range(1,8):
    fits_filename = fits_path + str(i) + "-dirty.fits"
    psf_filename = fits_path + str(i) + "-psf.fits"
    dirty_np = numpy.concatenate((dirty_np, fits.open(fits_filename)[0].data), axis = 1)
    psf_np = numpy.concatenate((psf_np, fits.open(psf_filename)[0].data), axis = 1)

hdu_dirty = fits.open(fits_filename)
hdu_dirty[0].data = dirty_np;
hdu_dirty.writeto('wsclean-MF-dirty.fits', overwrite=True)

hdul_psf = fits.open(psf_filename)
hdul_psf[0].data = psf_np;
hdul_psf.writeto('wsclean-MF-psf.fits', overwrite=True)

# Import fits as xarrays
dirty_mf = import_image_from_fits("wsclean-MF-dirty.fits")
psf_mf = import_image_from_fits("wsclean-MF-psf.fits")

In [ ]:
# The cellsize is derived from the wsclean command (scale/size)
# 1arcmin / 1024 = 1.5e-5 deg
cellsize=1.5e-5

Define deconvolution settings for Radler and Rascil deconvolution

In [ ]:
n_iterations = 500
clean_threshold=0.001
frac_threshold=0.001
window_shape_='quarter'
loop_gain=0.7
ms_scales= [0, 3, 10, 30] 


settings = rd.Settings()
settings.algorithm_type = rd.AlgorithmType.multiscale
settings.trimmed_image_width = dirty_mf.pixels.shape[2]
settings.trimmed_image_height = dirty_mf.pixels.shape[3]
settings.pixel_scale.x = cellsize
settings.pixel_scale.y = cellsize  
settings.minor_iteration_count = n_iterations
settings.threshold = clean_threshold
settings.minor_loop_gain = loop_gain
# Radler gives a better result with automatically calculated scales, hence the setting below is disabled
# settings.multiscale.scale_list = ms_scales

beam_size = 0.0


Deconvolve using RASCIL

In [ ]:
comp, residual = deconvolve_cube(dirty_mf, psf_mf, niter=n_iterations, threshold=clean_threshold, fractional_threshold=frac_threshold,
                                  gain=loop_gain, scales=ms_scales, algorithm='mfsmsclean')

restored = restore_cube(comp, psf_mf, residual)


Deconvolve using RADLER

In [ ]:
psf_radler = psf_mf.pixels.to_numpy().astype(numpy.float32).squeeze()
dirty_radler = dirty_mf.pixels.to_numpy().astype(numpy.float32).squeeze()
restored_radler = numpy.zeros_like(dirty_radler)

radler_object = rd.Radler(settings, psf_radler, dirty_radler, restored_radler,
                          beam_size, rd.Polarization.stokes_i)
reached_threshold = False
reached_threshold = radler_object.perform(reached_threshold, 0)


Compare output images

In [ ]:

pylab.rcParams['figure.figsize'] = (30.0, 20.0)

residual_rascil = residual.pixels.to_numpy().astype(numpy.float32).squeeze()
restored_rascil = restored.pixels.to_numpy().astype(numpy.float32).squeeze()

residual_min = numpy.min([numpy.min(residual_rascil) , numpy.min(dirty_radler)])
residual_max = numpy.max([numpy.max(residual_rascil) , numpy.max(dirty_radler)])

fig, ax = plt.subplots(4,8)
for i in range(8):
    
    ax[0,i].imshow(restored_rascil[i,:,:], cmap = 'turbo') 
    ax[0,i].title.set_text('RASCIL: restored')
    ax[1,i].imshow(restored_radler[i,:,:], cmap = 'turbo')
    ax[1,i].title.set_text('RADLER: restored')  
    ax[2,i].imshow(residual_rascil[i,:,:], vmin=residual_min, vmax=residual_max, cmap = 'turbo')
    ax[2,i].title.set_text('RASCIL: residual')
    ax[3,i].imshow(dirty_radler[i,:,:], vmin=residual_min, vmax=residual_max, cmap = 'turbo')
    ax[3,i].title.set_text('RADLER: residual')
